In [1]:
import pandas as pd
import numpy as np

In [2]:
month = "Dec'23"
prev_month = "Nov'23"
two_months_ago = "Oct'23"
three_months_ago = "Sep'23"

In [3]:
master_calling = pd.read_excel("PL Loan Book Jan'24.xlsx", header=2)

In [4]:
master_calling.head()

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,...,May'23 - Repaid,Jun'23 - Repaid,July'23 - Repaid,Aug'23 - Repaid,Sep'23 - Repaid,Unnamed: 169,Unnamed: 170,Unnamed: 171,Unnamed: 172,Unnamed: 173
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-31,1.0
1,e85b616e-e54b-4d67-b929-5f4b612404e2,188ec580-5b20-435b-8b02-b1fd0c439c6b,liquiloans,quikr,500342,2022-01-31,2,2022,8765513171,2022-01-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-31,2.0
2,7f5aee97-5a17-421a-a94d-7db9c244265a,ad42040e-f906-451f-9fb7-51e7806f64a7,liquiloans,quikr,505767,2022-01-31,2,2022,8368166817,2022-01-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-28,3.0
3,74d77339-a17e-4f57-8250-27f457d4be39,a1efbe44-4f49-4615-bd61-fcd02c3aa299,liquiloans,quikr,509929,2022-01-31,2,2022,8437737918,2022-01-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-31,4.0
4,3515f18c-f39c-4faa-a6de-f0a3675922fe,b99f37fa-fbb7-4fdc-b555-565efd93a6ac,liquiloans,quikr,514338,2022-01-31,2,2022,8898663866,2022-01-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-30,5.0


In [5]:
#State
#City
#Tier (city tier)
#Repeat Cases (TopUps?)

#### EMI date Clearance type

In [6]:
master_calling[month + " clearance status"] = np.where(master_calling[month + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[month + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[month + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[month + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[month + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[month + " Presentation"])))))

master_calling[prev_month + " clearance status"] = np.where(master_calling[prev_month + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[prev_month + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[prev_month + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[prev_month + " Presentation"])))))
                                                
master_calling[two_months_ago + " clearance status"] = np.where(master_calling[two_months_ago + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[two_months_ago + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[two_months_ago + " Presentation"])))))

master_calling[three_months_ago + " clearance status"] = np.where(master_calling[three_months_ago + " Presentation"].isin(['Cleared']), 'A) Cleared',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['Bounced', 'Bounced (DPD)', 'NPA (unsettled)', 'NPA']), 'B) Bounce',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['T-BOUNCE']), 'C) T-Bounce', #Technical Bounce
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['Advance']), 'D) Advance',
                                                np.where(master_calling[three_months_ago + " Presentation"].isin(['FORECLOSED', 'Closed', 'Settled', 'Foreclosed']), 'E) Closed',master_calling[three_months_ago + " Presentation"])))))

#### Pre presentation bucket

In [7]:
master_calling[prev_month + " DPD"].unique()

array(['-', 'NPA', 'NPA (Unsettled)', 1, '30+', '60+', 12, 20, 9, '1-29+',
       'Settled', '90+', 3, 4, 6, 5, 8, 7, 2, 26, 28, 13, 'Closed', 11,
       16, 18, 21, 15, 27, 22, 19, 25, 10, 14, 23,
       'Settled (partial paid)', 24, 17, 0, -7, 'Foreclosed'],
      dtype=object)

In [8]:
((pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') >= 0) & (pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') < 29)).sum()

3553

In [9]:
master_calling[month + " Presentation bucket"] = np.where(master_calling[prev_month + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[prev_month + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[prev_month + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[prev_month + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[prev_month + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[prev_month + " DPD"].isin(['90+']), 'E) Bkt 4',
                                                np.where(master_calling[prev_month + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'F) Bkt 4+',
                                                np.where(master_calling[prev_month + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'G) Closed',master_calling[prev_month + " DPD"])))))))

master_calling[prev_month + " Presentation bucket"] = np.where(master_calling[two_months_ago + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[two_months_ago + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[two_months_ago + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['90+']), 'E) Bkt 4',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'F) Bkt 4+',
                                                np.where(master_calling[two_months_ago + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'G) Closed',master_calling[two_months_ago + " DPD"])))))))

master_calling[two_months_ago + " Presentation bucket"] = np.where(master_calling[three_months_ago + " DPD"].isin(['-']) | ((pd.to_numeric(master_calling[three_months_ago + " DPD"], errors='coerce') >= -45) & (pd.to_numeric(master_calling[three_months_ago + " DPD"], errors='coerce') <= 29)), 'A) Bkt 0',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['1-29+']), 'B) Bkt 1',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['30+']), 'C) Bkt 2',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['60+']), 'D) Bkt 3',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['90+']), 'E) Bkt 4',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['NPA (Unsettled)', 'NPA']), 'F) Bkt 4+',
                                                np.where(master_calling[three_months_ago + " DPD"].isin(['Settled', 'Closed', 'Settled (partial paid)', 'Foreclosed']), 'G) Closed',master_calling[three_months_ago + " DPD"])))))))


### TAT calculation

In [10]:
prev_month_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=prev_month)
two_months_ago_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=two_months_ago)
three_months_ago_df = pd.read_excel("Repayment_data_Sep'23 to Dec'23.xlsx", sheet_name=three_months_ago)

In [11]:
prev_month_df['EMI Submission date'] = pd.to_datetime(prev_month_df['EMI Submission date'])
prev_month_df['Collection Date'] = pd.to_datetime(prev_month_df['Collection Date'], errors='coerce')

two_months_ago_df['EMI Submission date'] = pd.to_datetime(two_months_ago_df['EMI Submission date'])
two_months_ago_df['Collection Date'] = pd.to_datetime(two_months_ago_df['Collection Date'], errors='coerce')

three_months_ago_df['EMI Submission date'] = pd.to_datetime(three_months_ago_df['EMI Submission date'])
three_months_ago_df['Collection Date'] = pd.to_datetime(three_months_ago_df['Collection Date'], errors='coerce')

In [12]:
prev_month_df[prev_month + ' TAT'] = np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(prev_month_df['Bounce Paid/unpaid'].isin(['Paid']),(prev_month_df['Collection Date'] - prev_month_df['EMI Submission date']).dt.days,
                        np.nan))

two_months_ago_df[two_months_ago + ' TAT'] = np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(two_months_ago_df['Bounce Paid/unpaid'].isin(['Paid']),(two_months_ago_df['Collection Date'] - two_months_ago_df['EMI Submission date']).dt.days,
                        np.nan))

three_months_ago_df[three_months_ago + ' TAT'] = np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Unpaid']), 999,
                        np.where(three_months_ago_df['Bounce Paid/unpaid'].isin(['Paid']),(three_months_ago_df['Collection Date'] - three_months_ago_df['EMI Submission date']).dt.days,
                        np.nan))

prev_month_df[prev_month + ' TAT_bkt'] = np.where(prev_month_df[prev_month + ' TAT'] < -60, np.nan,
                                            np.where(prev_month_df[prev_month + ' TAT'] <= -10, 'A) <= -10',
                                            np.where(prev_month_df[prev_month + ' TAT'] < 0, 'B) -10 to -1',
                                            np.where(prev_month_df[prev_month + ' TAT'] == 0, 'C) 0',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 4, 'D) 1-4',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 10, 'E) 4-10',
                                            np.where(prev_month_df[prev_month + ' TAT'] <= 29, 'F) 10 - 29',
                                            np.where(prev_month_df[prev_month + ' TAT'] == 999, 'G) Unpaid',prev_month_df[prev_month + ' TAT']))))))))

two_months_ago_df[two_months_ago + ' TAT_bkt'] = np.where(two_months_ago_df[two_months_ago + ' TAT'] < -60, np.nan,
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= -10, 'A) <= -10',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] < 0, 'B) -10 to -1',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] == 0, 'C) 0',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 4, 'D) 1-4',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 10, 'E) 4-10',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] <= 29, 'F) 10 - 29',
                                            np.where(two_months_ago_df[two_months_ago + ' TAT'] == 999, 'G) Unpaid',two_months_ago_df[two_months_ago + ' TAT']))))))))

three_months_ago_df[three_months_ago + ' TAT_bkt'] = np.where(three_months_ago_df[three_months_ago + ' TAT'] < -60, np.nan,
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= -10, 'A) <= -10',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] < 0, 'B) -10 to -1',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] == 0, 'C) 0',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 4, 'D) 1-4',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 10, 'E) 4-10',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] <= 29, 'F) 10 - 29',
                                            np.where(three_months_ago_df[three_months_ago + ' TAT'] == 999, 'G) Unpaid',three_months_ago_df[three_months_ago + ' TAT']))))))))

In [13]:
master_calling.head(2)

,niro_user_id,Niro Opportunity ID,Disbursal Partner Name,Platform Partner Name,Lender Loan Id,Disbursement Month,Month Number,Disbursement Year,Phone Numbers,Disbursement Date,...,Unnamed: 171,Unnamed: 172,Unnamed: 173,Dec'23 clearance type,Nov'23 clearance status,Oct'23 clearance status,Sep'23 clearance status,Dec'23 Presentation bucket,Nov'23 Presentation bucket,Oct'23 Presentation bucket
0,0a2c0c4e-e0bf-4e50-909e-25ad6272119e,0f100756-11ae-4472-bd29-4fdbd6c37fdb,liquiloans,quikr,499217,2021-12-31,1,2021,9900368067,2021-12-31,...,NaN,2021-12-31,1.0,E) Closed,E) Closed,E) Closed,E) Closed,A) Bkt 0,A) Bkt 0,A) Bkt 0
1,e85b616e-e54b-4d67-b929-5f4b612404e2,188ec580-5b20-435b-8b02-b1fd0c439c6b,liquiloans,quikr,500342,2022-01-31,2,2022,8765513171,2022-01-04,...,NaN,2022-01-31,2.0,E) Closed,E) Closed,E) Closed,E) Closed,A) Bkt 0,A) Bkt 0,A) Bkt 0


In [14]:
prev_month_df.head(1)

,Opportunity ID,Loan Id,EMI Submission date,Bounced cases,Bounce Paid/unpaid,Collection Date,DPD,Nov'23 TAT,Nov'23 TAT_bkt
0,0f100756-11ae-4472-bd29-4fdbd6c37fdb,499217,2023-11-02,Closed,Closed,NaT,0,NaN,nan


In [15]:
master_calling = master_calling.merge(prev_month_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", prev_month + ' TAT', prev_month + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") \
    .merge(two_months_ago_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", two_months_ago + ' TAT', two_months_ago + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") \
    .merge(three_months_ago_df.rename(columns = {'Opportunity ID':'Niro Opportunity ID'})[["Niro Opportunity ID", three_months_ago + ' TAT', three_months_ago + ' TAT_bkt']], on = "Niro Opportunity ID", how = "left") 

In [17]:
master_calling.to_excel("master_file_" + month + ".xlsx", index=False)